In [1]:
import pandas as pd

In [2]:
import constants
import constraints

from datagolf import datagolf
from fileman import fileman
from picklejar import PickleJar

In [3]:
constants.pandas_settings()
urls = {
    'skills-decomp': 'https://feeds.datagolf.com/preds/player-decompositions?tour={tour}&file_format={file_format}&key=6a626b6c312c0d33cfe157d614b5',
    'dfs-projections': 'https://feeds.datagolf.com/preds/fantasy-projection-defaults?tour={tour}&site={site}&slate={slate}&file_format={file_format}&key=6a626b6c312c0d33cfe157d614b5'
}

In [4]:
datagolf.load_skills().head(10)

,name,baseline,final,category-sg,true-sg,driving-accuracy,total-fit
0,Sungjae Im,1.10,1.42,0.22,0.07,0.10,0.05
1,Tony Finau,1.44,1.23,-0.04,-0.02,-0.05,-0.14
2,Hideki Matsuyama,1.35,1.21,-0.02,-0.04,0.02,-0.05
3,Cameron Tringale,0.83,0.87,-0.04,0.04,0.02,-0.02
4,Brendan Steele,0.64,0.81,0.17,-0.02,0.07,0.03
5,Adam Hadwin,0.76,0.81,-0.07,0.03,0.08,0.05
6,Maverick McNealy,0.87,0.80,-0.04,0.04,0.01,-0.07
7,Davis Riley,0.71,0.75,0.08,0.02,0.04,-0.02
8,Adam Long,0.62,0.69,-0.10,-0.02,0.13,0.16
9,Cameron Davis,0.63,0.65,0.05,0.06,-0.08,-0.18


In [9]:
skills=datagolf.load_skills(decomp=False) #.head(10)
skills['drv-acc'].describe()

count   119.00
mean      0.00
std       0.05
min      -0.12
25%      -0.03
50%      -0.00
75%       0.04
max       0.13
Name: drv-acc, dtype: float64

In [6]:
datagolf.load_dfs().head(10)

,name,salary,points,later,ownership
0,Tony Finau,12000.00,90.60,0.00,15.11
1,Hideki Matsuyama,11900.00,90.49,1.00,14.66
2,Sungjae Im,11500.00,95.02,1.00,17.06
3,Adam Hadwin,11400.00,81.31,1.00,14.78
4,Maverick McNealy,11300.00,82.43,1.00,15.03
5,Sahith Theegala,11200.00,77.04,1.00,15.52
6,Davis Riley,11100.00,81.07,0.00,21.39
7,Cameron Tringale,11000.00,82.85,0.00,16.29
8,Cameron Davis,10900.00,79.29,0.00,13.67
9,J.T. Poston,10800.00,77.85,0.00,9.36


In [7]:
datagolf.load_baselines().head(10)

,name,make-cut,1st(%),2nd(%),3rd(%),top-10(%),top-20(%)
0,Sungjae Im,0.82,0.07,0.12,0.16,0.36,0.52
1,Hideki Matsuyama,0.81,0.06,0.11,0.15,0.33,0.50
2,Tony Finau,0.80,0.05,0.10,0.14,0.32,0.49
3,Maverick McNealy,0.75,0.03,0.06,0.09,0.24,0.39
4,Brendan Steele,0.73,0.03,0.06,0.09,0.22,0.37
5,Sahith Theegala,0.74,0.03,0.05,0.07,0.21,0.36
6,Cameron Tringale,0.73,0.03,0.05,0.07,0.21,0.36
7,Davis Riley,0.73,0.03,0.05,0.08,0.21,0.36
8,Cameron Davis,0.72,0.03,0.05,0.08,0.21,0.35
9,Adam Hadwin,0.72,0.03,0.05,0.07,0.20,0.34
